# Performing raster stats on remote datasets

## Scenario
Your uncle is moving to Tucson from New York City and he is deathly afraid of all the desert creatures he has heard about. He has asked you to help him find a neighborhood that is well-lit so that he can be sure to see any venomous creatures which might be lurking about. You googled 'well-lit neighborhoods in Tucson' but nothing came up. Luckily, you are a GIS professional and you know exactly what to do. With a neighborhoods polygon and a streetlights dataset you can simply do some zonal stats and figure out how much ambient light there is in each neighborhood and simply recommend those to your uncle and his realtor.


## Overview
This Jupyter Notebooks walks you through some basic spatial analysis using the `geopandas`, `numpy`, and `rasterstats` Python libraries.

In this tutorial you will learn how to:

- acquire spatial data from a remote resource as a GeoPandas `DataFrame`
- inspect a GeoPandas `DataFrame`
- visualize a GeoPandas `DataFrame`
- replace null values in a `DataFrame` attribute table
- create a heat map from points dataset using `numpy`
- create a raster (GeoTIFF) from scratch using a numpy array
- perform zonal stats using a raster and a vector with the `rasterstats` library
- sort a GeoPandas `DataFrame`

The data you will be using will be from the City of Tucson Open GIS Data website: 
- (Streetlights)[http://gisdata.tucsonaz.gov/datasets/streetlights-city-of-tucson-open-data]
- (Neighborhood Associations)[http://data-cotgis.opendata.arcgis.com/datasets/neighborhood-associations-open-data]

Don't download the data now; you will do that programmatically below.

## Install python libraries
Your jupyter environment (aka `kernel` is unlikely to have these libraries installed so you will have to install them yourself. It will be easier to do this outside the notebook where you can see the output and debug any issues if necessary

Run this from your `anaconda shell`:
```
conda install --yes -c conda-forge rasterstats requests scipy rasterio numpy pandas geopandas matplotlib descartes proj4==5.2.0
```


In [ ]:
import rasterio
import math
import requests
import os
import scipy.ndimage.filters
import rasterio as rio
from rasterio.transform import from_origin
from rasterio.plot import show
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from rasterstats import zonal_stats

To display graphical plots in jupyter:

In [ ]:
%matplotlib inline

Let's open our first dataset; street lights. This data is available online from http://gisdata.tucsonaz.gov/datasets/streetlights-city-of-tucson-open-data. I've copied the GeoJSON link and added it below. We are going to first check to see if the file has already been downloaded and, if not, then download it using the `requests` library, and write it out to a file.

In [ ]:
url = 'https://opendata.arcgis.com/datasets/09ed59b6aae2483aa1bd32837d4aa7e5_19.geojson'
r = requests.get(url)
with open('data/street_lights.geojson', 'w') as f:
    f.write(r.content.decode("utf-8"))


We just downloaded the file above. Now let's load it using the `geopandas` library, which knows how to interpret its spatial nature and its attributes. Once we open it, we'll take a peek at the first 5 lines using the `head()` function that pandas provides. You can always use that method when you have a `DataFrame` or `GeoDataFrame` to see the first few lines of the table.

In [ ]:
street_lights=gpd.read_file('data/street_lights.geojson')
street_lights.head()

It looks like we have attribute data, but what about the spatial part? Let's make a crude graph using the `matplotlib` library.

In [ ]:
fig, ax = plt.subplots(figsize = (10,10))
street_lights.plot(ax=ax)
plt.show()

We are going to ultimately make a heatmap using the `Wattage` field but it is going to create some undesirable artifacts if there are any `NaN` fields (`NaN` means "not a number" for numeric fields). The `numpy` library provides a function to get a list of the indices matching a certain condition. In this case, the condition is where the fields are `NaN`. 

What will happen if we don't clean those up? When we try to create a heatmap, the function will be applied to all the values in a certain neighborhood. If that function is, say, `mean`, and it tries to take the mean of `1`, `2`, and `NaN`, the answer will also be `NaN`. We want to replace them all with `0` (you can use another realistic value if you'd like).

In [ ]:
inds = np.where(street_lights['Wattage'].isnull())
inds

To see that the `Wattage` field is actually full of those `NaN` values, let's look at a slice with those indexes:

In [ ]:
street_lights['Wattage'][inds[0]]

This is a very common issue! `Pandas` provides a convenient way of fixing those by replacing them with a set value. Use the `fillna` function and replace the values in place. Then, print the indices of the null values afterwards as a sanity check.

In [ ]:
street_lights['Wattage'].fillna(value=0, inplace = True)
inds = np.where(street_lights['Wattage'].isnull())
inds

Great! You now have a clean street lights dataset. Our next task is to create a heat map. To do this we are going to construct an array that will contain all the raster values over our area, then create a raster from scratch based on the bounds of our street_lights data. You can experiment with different kernel shapes below and see how that affects the output. In the meantime, let's extract the spatial coordinates of the points dataset.

In [ ]:
x = street_lights['Spatial_X']
y = street_lights['Spatial_Y']
minx = min(x)
maxx = max(x)
miny = min(y)
maxy = max(y)

Calculate the number of cells in the x and y direction. Since this is a geographic coordinate system and we want our cells about the same dimensions in x and y, we scale the xcount based on the cosine of the latitude of Tucson (about 0.846). If you want a raster that is more or less dense than the example below, modify the `cell_size_y` value.

In [ ]:
cell_size_y=0.001
ycount = math.ceil((maxy-miny)/cell_size_y)
xcount = math.ceil(ycount/math.cos(math.radians(32.2)))
xsize = (maxx-minx)/xcount
ysize = (maxy-miny)/ycount

Brighter lights contribute more to the ambient light than softer lights so we are going to use `Wattage` to further weight for each point. 

In [ ]:
weights = street_lights['Wattage'].copy()

Finally, we summarize the street lights by each grid cell, adding their weights within each cell. Since we are adding a bunch of lights in some cells and the units are `Watts`, these summary values will be quite large and not have an easily interpretable meaning. It might be something along the lines of `cumulative streetlight watts per cell`

In [ ]:
h, _, _ = np.histogram2d(x, y, bins=(np.linspace(minx, maxx, xcount), np.linspace(miny, maxy, ycount)), weights=weights)
type(h)
h.shape

The lights have an effect beyond the cell they reside in. We will ignore the physics about light decay and just assume that there is a local neighborhood effect and describe it by a `Gaussian` filter. You can play with different filters and different filter parameters to see how they affect the resulting heatmap. There are a _lot_ of filter options that you can read about on the (SciPy.org)[https://docs.scipy.org/doc/scipy/reference/ndimage.html] webpage.

In [ ]:
z = scipy.ndimage.filters.gaussian_filter(h.T, 1)

We ran the filter. Just a sanity check to see that we still have the same type of thing

In [ ]:
type(z)
z.shape

The `numpy` library also has a way of easily finding the max or min of a dataset. 

In [ ]:
np.amax(z)

Don't worry! That's to be expected. Edge effects means the filter had to reach outside the source data to calcualte the new value and it filled them in with `NaN`s which, as you know, will cause the resulting function to also evaluate to `NaN`. We don't need to fix that data but we did want to see if the filter actually produced real data. `numpy` provides an alternative `max` function that ignores `NaN` since it is such a common use case.

In [ ]:
np.nanmax(z)

The next step is to construct the raster metadata for us to put the Gaussian-smoothed 2-d histogram data in. We set the `transform` which is really just the upper left coordinate and cell sizes (similar to what you get in a tif or jpg world file), the projection, the number of cells, the data type, and the number of bands.

In [ ]:
transform = from_origin(minx, maxy, xsize, ysize)
meta = { 'count':1, # number of bands
         'dtype': z.dtype, # copy the data type from the filter output
         'driver': 'GTiff', # GeoTiff ftw
         'width': xcount, # number of cells
         'height': ycount, 
         'crs':rio.crs.CRS.from_dict(init='epsg:4326'), # WGS84 lat/long coordinate reference system
         'transform': transform}

Finally, write that heatmap to a file!

In [ ]:
with rio.open('data/streetlights_heatmap.tif', 'w', **meta) as dst:
    dst.write(z, 1)

To get a very quick sanity check that the heatmap has believable data we'll use the `rasterio` package to load and view:

In [ ]:
raster = rio.open('data/streetlights_heatmap.tif')
show(raster)

We now have a heatmap so it's time to load our second dataset, a polygon dataset. We will inspect it as well.

In [ ]:
url = 'https://opendata.arcgis.com/datasets/828d637891e94d95a2e62cf62ad2f7e0_0.geojson'
r = requests.get(url)
with open('data/nbrhoods.geojson', 'w') as f:
    f.write(r.content.decode("utf-8"))
    

In [ ]:
nbrhoods=gpd.read_file('data/nbrhoods.geojson')
nbrhoods.head()

In [ ]:
fig, ax = plt.subplots(figsize = (10,10))
nbrhoods.plot(ax=ax)
plt.show()

We have our final input datasets for running zonal stats. Zonal stats runs on the files. You will get a warning about `nodata` which you can safely ignore.

In [ ]:
zs = zonal_stats('data/nbrhoods.geojson', 'data/streetlights_heatmap.tif')

Inspect the zonal stats output. You will see it is an array of dictionaries.

In [ ]:
zs

In [ ]:
type(zs)

In [ ]:
type(zs[0])

In [ ]:
zs[0]

You may also recognize that there are no feature IDs in the dict. Fortunately, the stats records are in the same natural order as the records in the polygon input.

In [ ]:
len(zs)

In [ ]:
len(nbrhoods)

We want to add a new column to the zonal stats but it isn't as simple as the `mean`. We want to calculate the `mean` over `area`. This is also a good opportunity to point out what's called a `list comprehension`, which is a concise way of populating a list. Since the values we want are in a `dict` inside a `list` we need to extract them, or promote them to their own list. To do this:

In [ ]:
light_mean = [rec['mean'] for rec in zs]

Next, let's divide the zonal mean by the polygon area to get the average light over the neighborhood.

In [ ]:
light_mean_per_area = light_mean / nbrhoods.area

Finally, we can expand the `nbrhoods` `GeoDataFrame` simply by declaring a dict entry and giving it the value or our list:

In [ ]:
nbrhoods['light_avg'] = light_mean_per_area

How does it look? Let's take a peek and we will set the display column to `light_avg` and let the underlying `plot` logic build a chloropleth map for us.

* Save this photo as light_avg.png as a deliverable for your assignment *

In [ ]:
nbrhoods.plot(column='light_avg', legend=True)

It looks believable. Of course you can open these files in QGIS and inspect them closer, but assuming they pass the test, it's time to sort the neighborhoods based on their average light and help out your uncle with some suggestions.

In [ ]:
nbrhoods.sort_values(by='light_avg', ascending=False)

To finish up, save the `nbrhoods` `GeoDataFrame` to a shapefile:

In [ ]:
nbrhoods.to_file('data/nbrhoods.shp')

# Assignment Deliverables

Create a new branch with the following and submit a Pull Request to merge with `master`:

1. Add `light_avg.png` from above
The `light_avg.png` image from above should be added to this repo (in a new branch of course)

2. Modify notebook and run again
Your uncle has moved to Tucson and and has become a vocal advocate for public safety. After almost getting stuck in an urban wash during monsoon season, he has a plan to improve the water safety barricades and wants to run a pilot project in the city ward with the most water barricades. Use the data from the City of Tucson's Open GIS Data and produce a map of the wards, color-coded by the number of water safety barricades. Use the data for [City Wards](http://data-cotgis.opendata.arcgis.com/datasets/city-of-tucson-wards-open-data) and [Operation Splash](http://data-cotgis.opendata.arcgis.com/datasets/operation-splash-open-data). As a deliverable, submit:

- chloropleth map showing barricades by ward
- screenshot showing table of the number of barricades by ward